In [ ]:
from functools import cache
from pathlib import Path
import numpy as np


RAW_FLUX_DATA_PATH: Path = (
    Path(".").resolve().parent.parent.parent.parent / "data" / "flux" / "raw"
)
FLUXTRACE_FILENAME_CONVENTION: str = "fluxes_{iteration}.dat"
flux_trace_filenames: list[Path] = list(
    RAW_FLUX_DATA_PATH.glob(FLUXTRACE_FILENAME_CONVENTION.format(iteration="*"))
)
NR_FLUX_TRACES: int = len(flux_trace_filenames)
print(f"Number of flux traces found: {NR_FLUX_TRACES}")


@cache
def load_flux_data(idx: int) -> np.ndarray:
    file_path: Path = RAW_FLUX_DATA_PATH / FLUXTRACE_FILENAME_CONVENTION.format(
        iteration=idx
    )
    data: np.ndarray = np.loadtxt(file_path)
    return data[:, 1]  # only energy flux for now, ignore particle and momentum fluxes

In [ ]:
# filter invalid flux traces
valid_flux_trace_indices: list[int] = []
HORIZON: int = 240

for idx in range(NR_FLUX_TRACES):
    flux_data: np.ndarray = load_flux_data(idx)
    mean_head: float = float(np.mean(flux_data[:HORIZON]))
    mean_tail: float = float(np.mean(flux_data[-HORIZON:]))
    if not (1.0 <= mean_head <= np.inf) or not (1.0 <= mean_tail <= np.inf):
        continue
    valid_flux_trace_indices.append(idx)

print(f"Number of valid flux traces: {len(valid_flux_trace_indices)}")
print(
    f"Valid flux trace indices under 100: {len([i for i in valid_flux_trace_indices if i < 100])}"
)

In [ ]:
# subsampling every third timestep
SUBSAMPLE_FACTOR: int = 3
subsampled_flux_traces: dict[int, np.ndarray] = {}

for idx in valid_flux_trace_indices:
    flux_data: np.ndarray = load_flux_data(idx)
    subsampled_flux_data: np.ndarray = flux_data[::SUBSAMPLE_FACTOR]
    subsampled_flux_traces[idx] = subsampled_flux_data

print(
    f"Mean length of subsampled flux traces: {np.mean([len(trace) for trace in subsampled_flux_traces.values()])}"
)

In [ ]:
import matplotlib.pyplot as plt

means: list[float] = [
    float(np.mean(trace)) for trace in subsampled_flux_traces.values()
]
plt.hist(means, bins=50)
plt.xlabel("Mean flux value")
plt.ylabel("Number of traces")
plt.title("Histogram of Mean Flux Values Across Subsampled Traces")
plt.show()